## Recurrent Neural Networks

In this assignment, we will learn about recurrent neural networks. We will create an RNN and learn to classify text data.

In [1]:
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd

In [2]:
yelp = pd.read_csv('https://tf-assets-prod.s3.amazonaws.com/tf-curric/data-science/yelp_labeled.csv', error_bad_lines=False)

b'Skipping line 281: expected 2 fields, saw 3\nSkipping line 290: expected 2 fields, saw 3\nSkipping line 296: expected 2 fields, saw 3\nSkipping line 322: expected 2 fields, saw 3\nSkipping line 373: expected 2 fields, saw 3\nSkipping line 417: expected 2 fields, saw 3\nSkipping line 427: expected 2 fields, saw 3\nSkipping line 429: expected 2 fields, saw 3\nSkipping line 577: expected 2 fields, saw 3\nSkipping line 578: expected 2 fields, saw 3\nSkipping line 611: expected 2 fields, saw 3\nSkipping line 677: expected 2 fields, saw 3\nSkipping line 771: expected 2 fields, saw 3\nSkipping line 930: expected 2 fields, saw 3\nSkipping line 979: expected 2 fields, saw 4\nSkipping line 980: expected 2 fields, saw 3\n'


In [3]:
yelp

,text,sentiment
0,Wow... Loved this place.,1
1,Crust is not good.,0
2,Not tasty and the texture was just nasty.,0
3,Stopped by during the late May bank holiday of...,1
4,The selection on the menu was great and so wer...,1
...,...,...
971,I think food should have flavor and texture an...,0
972,Appetite instantly gone.,0
973,Overall I was not impressed and would not go b...,0
974,The whole experience was underwhelming and I t...,0


In [4]:
yelp.sentiment.value_counts()

1    494
0    482
Name: sentiment, dtype: int64

We have loaded a Yelp review dataset above. A positive sentiment is classified as 1 and a negative sentiment is classified as 0. 

In [5]:
from nltk.corpus import stopwords
import re
from nltk.stem.porter import PorterStemmer

stemmer = PorterStemmer()

def remove_stopwords(input_text):
        stopwords_list = stopwords.words('english')
        # Some words which might indicate a certain sentiment are kept via a whitelist
        whitelist = ["n't", "not", "no"]
        words = input_text.split() 
        clean_words = [word for word in words if (word not in stopwords_list or word in whitelist) and len(word) > 1] 
        return " ".join(clean_words)       

def stem_list(word_list):
    stemmed = []
    for word in word_list:
        stemmedword = stemmer.stem(word)
        stemmed.append(stemmedword)
    return stemmed

def normalize(terms):
    terms = terms.lower()
    terms = remove_stopwords(terms)
    word_delimiters = u'[\\[\\]\n.!?,;:\t\\-\\"\\(\\)\\\'\u2019\u2013 ]'
    term_list = re.split(word_delimiters, terms)
    trimmed = [x.rstrip() for x in term_list]
    stemmed = stem_list(trimmed)
    space = ' '
    normed = space.join(stemmed)
    normed = normed.replace('  ', ' ')
    return normed

In the code block above, we have functions to remove stopwords, stem, and normalize the text (remove special characters and trim white space). Apply the normalize function to every yelp review and assign the normalized text to a new column.

In [6]:
# Answer below:
import nltk
nltk.download('stopwords')
nltk.download('punkt')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [7]:
yelp['normalized_text'] = yelp['text'].map(lambda x: normalize(x))

Next, use the one hot function for text encoding and encode the normalized text. Determine the vocabulary size to perform the encoding.

In [8]:
from nltk import word_tokenize

words = []
for doc in yelp['normalized_text']:
  for word in word_tokenize(doc):
      words.append(word)

unique = set(words)
nb_unique_words = len(unique)
nb_unique_words

1626

In [9]:
# Answer below:
from tensorflow.keras.preprocessing.text import one_hot

vocab_size = nb_unique_words
yelp['encoded'] = yelp['normalized_text'].apply(lambda x: one_hot(x, vocab_size))


Convert the encoded sequences into a numpy array and make sure all reviews are the same length using the `pad_sequences` function in Keras.

In [10]:
# Answer below:
from tensorflow.keras.preprocessing.sequence import pad_sequences

encoded = np.array(pad_sequences(yelp['encoded']))

Split the data into train and test. Use 20% for test. The sentiment column should be used as the target variable.

In [11]:
# Answer below:
from sklearn.model_selection import train_test_split

X = encoded
y = yelp['sentiment']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [12]:
len(X_train[0])

83

In [13]:
len(X_test[0])

83

Create a sequential model. The model should contain an embedding layer with input dim that is the size of the largest encoding in the vocabulary. The output dim should be 100, the input length is the number of columns in the training data. 
After the embedding layer, add a SimpleRNN layer with unit size 32, a dense layer of size 8 and a dense output layer.

In [14]:
# Answer below:
from tensorflow.keras.models import Sequential 
from tensorflow.keras.layers import Dense, Embedding, SimpleRNN

max_words = nb_unique_words # grabs the largest numeric encoded value

model = Sequential()
model.add(Embedding(max_words, output_dim=100, input_length=X_train.shape[1]))
model.add(SimpleRNN(32))
model.add(Dense(8, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 83, 100)           162600    
_________________________________________________________________
simple_rnn (SimpleRNN)       (None, 32)                4256      
_________________________________________________________________
dense (Dense)                (None, 8)                 264       
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 9         
Total params: 167,129
Trainable params: 167,129
Non-trainable params: 0
_________________________________________________________________


Compile using the optimizer of your choice, use crossentropy for your loss function. Fit the model using a batch size of 128 and 50 epochs

In [15]:
# Answer below:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [16]:
model.fit(X_train, y_train, epochs=50, batch_size=128, validation_data=(X_test, y_test))

Epoch 1/50
7/7 [==============================] - 2s 115ms/step - loss: 0.6956 - accuracy: 0.4857 - val_loss: 0.6907 - val_accuracy: 0.5204
Epoch 2/50
7/7 [==============================] - 0s 37ms/step - loss: 0.6747 - accuracy: 0.7516 - val_loss: 0.6872 - val_accuracy: 0.5612
Epoch 3/50
7/7 [==============================] - 0s 38ms/step - loss: 0.6511 - accuracy: 0.7856 - val_loss: 0.6857 - val_accuracy: 0.5510
Epoch 4/50
7/7 [==============================] - 0s 36ms/step - loss: 0.6210 - accuracy: 0.8626 - val_loss: 0.6733 - val_accuracy: 0.6276
Epoch 5/50
7/7 [==============================] - 0s 35ms/step - loss: 0.5725 - accuracy: 0.9019 - val_loss: 0.6988 - val_accuracy: 0.5153
Epoch 6/50
7/7 [==============================] - 0s 36ms/step - loss: 0.5363 - accuracy: 0.7801 - val_loss: 0.6652 - val_accuracy: 0.5408
Epoch 7/50
7/7 [==============================] - 0s 37ms/step - loss: 0.4807 - accuracy: 0.8668 - val_loss: 0.6296 - val_accuracy: 0.7194
Epoch 8/50
7/7 [==========